In [6]:
#creating sample of size 1000 to test csds on 

# importing package
import pandas as pd
import random
import csv


# file paths
filename = '/home/jovyan/ODBiz/6-AssignCSDs/Test/4-ODBiz_merged_2022-07-28-Copy1.csv'
newfile = '/home/jovyan/ODBiz/6-AssignCSDs/Test/Filtred_Sample.csv'



# filtering original merging csv 
data = pd.read_csv(filename, low_memory=False)                             # read the file in as a csv file
data = pd.DataFrame(data)                                                  # turn it into a data frame 
data = data.loc[(data['latitude'] > 0 ) & (data['longitude'] < 0 ) ]       # filter so that it only contains the rows with lat/long info 

data.to_csv(newfile, index=False)                                          # save new data frame as a csv  





#create sample data set 
n = sum(1 for line in open(newfile)) - 1                                  # number of records in file (excludes header)
s = 1000                                                                  # desired sample size = 1000
skip = sorted(random.sample(range(1,n+1),n-s))                            # the 0-indexed header will not be included in the skip list
df = pd.read_csv(newfile, skiprows=skip)

df = pd.DataFrame(df)


df.to_csv('/home/jovyan/ODBiz/6-AssignCSDs/Test/Sample3.csv', index=False)  

#df.to_csv('/home/jovyan/ODBiz/6-AssignCSDs/Test/Sample_Set_1000.csv', index=False)     # save to Sample Set file 

In [8]:
"""
this code was modified from the ODHF v2 project for the ODBiz project
"""

import pandas as pd
import geopandas as gpd

df=pd.read_csv("/home/jovyan/ODBiz/6-AssignCSDs/Test/Sample.csv", low_memory=False, dtype='str')

#df=pd.read_csv("/home/jovyan/ODBiz/6-AssignCSDs/Test/Sample_Set_1000.csv", low_memory=False, dtype='str')



gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']))
gdf.crs="EPSG:4326"


#read in Statcan boundary file
CSD = gpd.read_file("/home/jovyan/ODBiz/6-AssignCSDs/CSD_shapefile/lcsd000a16a_e.shx")
CSD=CSD[['CSDUID', 'CSDNAME','PRUID', 'geometry']]

#convert geometry of addresses to statcan geometry
gdf=gdf.to_crs(CSD.crs)


#perform spatial merge
gdf_csd=gpd.sjoin(gdf,CSD, predicate='within', how='left')

df=pd.DataFrame(gdf_csd)



df.to_csv('/home/jovyan/ODBiz/6-AssignCSDs/Test/Test_CSDs.csv', index=False)
df.to_csv('/home/jovyan/ODBiz/Deduplication/ODHF_copy/inputs/Test_CSDs.csv', index=False)